In [2]:
import re
from pprint import pprint

import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained("internlm/internlm2-math-7b", trust_remote_code=True)
# Set `torch_dtype=torch.float16` to load model in float16, otherwise it will be loaded as float32 and might cause OOM Error.
model = AutoModelForCausalLM.from_pretrained("internlm/internlm2-math-7b", trust_remote_code=True, torch_dtype=torch.float16).cuda()
model = model.eval()

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
response, history = model.chat(tokenizer, "1+1=", history=[], meta_instruction="")
print(response)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


1 + 1 = 2.


In [5]:
history

[('1+1=', '1 + 1 = 2.')]

Generate comments

In [16]:
prompt = "Output a list of 100 hints that would guide a proof. Use most common strategies and proof techniques to craft high quality, helpful comments. Give a mix of high-level and math technical comments."

In [14]:
response, history = model.chat(tokenizer, prompt, history=[], meta_instruction="")

In [15]:
pprint(response)

('1. Start with a clear statement of the theorem you want to prove.\n'
 '2. Make sure you understand the problem well and identify the key properties '
 'and definitions needed for the proof.\n'
 '3. Work with small cases or examples to gain intuition and develop a '
 'possible proof strategy.\n'
 '4. Decide on a proof strategy, such as direct proof, contradiction, '
 'induction, or construction.\n'
 '5. Use proof by contradiction when direct proof is difficult or impossible.\n'
 '6. Use induction when the statement is recursive or involves a set of '
 'elements.\n'
 '7. Use construction when the statement involves creating an object with '
 'specific properties.\n'
 '8. Keep your proof organized and use headings and paragraphs to break it up '
 'into logical steps.\n'
 '9. Use forward proof by assuming the goal and showing it implies the '
 'hypothesis.\n'
 '10. Use backward proof by assuming the hypothesis and showing it implies the '
 'goal.\n'
 '11. Use case analysis to break the p

In [17]:
prompt = "Output a list of 100 hints that would guide a theorem prover prove a olympiad math problem. Use most common strategies and proof techniques. Give a mix of high-level and math technical comments."

In [19]:
response, history = model.chat(tokenizer, prompt, history=[], meta_instruction="")
pprint(response)

('1. State the problem and goals: Clearly define the problem statement and the '
 'goals you are trying to achieve.\n'
 '2. Assume the opposite: Consider assuming the opposite of what you want to '
 'prove and see if you can reach a contradiction.\n'
 '3. Use proof by contradiction: If you can derive a contradiction from '
 'assuming the opposite, then you have proven the original statement to be '
 'true.\n'
 '4. Break down the problem: Try to break down the problem into smaller parts '
 'and prove each part separately.\n'
 '5. Use induction: If the problem involves a sequence or a set, consider '
 'using mathematical induction to prove the statement.\n'
 '6. Invariants: Identify any invariants that are preserved during the process '
 'and use them to simplify the problem.\n'
 '7. Draw a diagram or visualize the problem: Sometimes, drawing a diagram or '
 'visualizing the problem can help you gain insights and find a solution.\n'
 '8. Apply algebraic techniques: If the problem involve

Bad quality, let's use gpt o1-preview instead.

In [ ]:
# https://chatgpt.com/share/66f5a990-3c38-8008-aebd-691fba96ce72

Can InterML classify problem type?

In [20]:
prompt = """ Is the type of the problem this lean proof solves algebra, number theory, combinatorics, or geometry?
/-!
# International Mathematical Olympiad 1963, Problem 5

Prove that cos(π/7) - cos(2π/7) + cos(3π/7) = 1/2.
-/

namespace Imo1963P5

open scoped Real

problem imo1963_p5 :
    Real.cos (π/7) - Real.cos (2*π/7) + Real.cos (3*π/7) = 1/2 := by
  rw [show (2*π/7) = π - (5*π/7) by linarith]
  rw [Real.cos_pi_sub]
  simp only [sub_neg_eq_add]
  have h : 2 * Real.sin (π / 7) ≠ 0 := by
    simp only [ne_eq, mul_eq_zero, OfNat.ofNat_ne_zero, false_or]
    apply ne_of_gt
    apply Real.sin_pos_of_pos_of_lt_pi
    simp only [Nat.ofNat_pos, div_pos_iff_of_pos_right, Real.pi_pos]
    trans 1
    · rw [div_lt_one (by linarith only)]
      linarith only [Real.pi_le_four]
    · linarith only [Real.pi_gt_three]
  apply (mul_right_inj' h).mp
  rw [left_distrib, left_distrib]
  have prod_sum : ∀ (x y : ℝ),
      2 * Real.sin x * Real.cos y = Real.sin (x + y) - Real.sin (y - x) := by
    intro x y
    rw [Real.sin_add, Real.sin_sub]
    linarith only
  rw [prod_sum, prod_sum, prod_sum]
  rw [show (π / 7 + π / 7)     = 2 * π / 7 by linarith only]
  rw [show (π / 7 - π / 7)     = 0         by linarith only]
  rw [show (π / 7 + 5 * π / 7) = 6 * π / 7 by linarith only]
  rw [show (5 * π / 7 - π / 7) = 4 * π / 7 by linarith only]
  rw [show (π / 7 + 3 * π / 7) = 4 * π / 7 by linarith only]
  rw [show (3 * π / 7 - π / 7) = 2 * π / 7 by linarith only]
  rw [Real.sin_zero]
  ring_nf
  rw [← Real.sin_pi_sub]
  rw [show (π - π * (6 / 7)) = π / 7 by linarith]
  congr
  linarith
end Imo1963P5
"""

In [21]:
response, history = model.chat(tokenizer, prompt, history=[], meta_instruction="")
pprint(response)

'The type of the problem this lean proof solves is algebra.'


nice. Now gather all types of comments into `comments.txt`.

In [28]:
# Define file paths
gpt_comments_path = 'data/prover-llm_v0/comments/gpt-comments-filtered.txt'
observations_path = 'data/prover-llm_v0/comments/observations.txt'
output_path = 'data/prover-llm_v0/comments/comments.txt'

# Function to clean up the observations file
def clean_observations(content):
    # Remove headers like **Algebra**, **Number Theory**, etc.
    content = re.sub(r'\*\*[A-Za-z\s]+\*\*', '', content)
    
    # Remove numbering (like "1. ", "2. ", etc.)
    content = re.sub(r'^\d+\.\s+', '', content, flags=re.MULTILINE)
    
    # Remove extra blank lines but keep a single newline separation
    content = re.sub(r'\n\s*\n+', '\n', content).strip()
    
    return content

# Read the contents of both files
with open(gpt_comments_path, 'r') as file1, open(observations_path, 'r') as file2:
    gpt_comments = file1.read().strip()
    observations = file2.read()

# Clean the observations
cleaned_observations = clean_observations(observations)

# Combine the contents, ensuring they are separated by a newline
combined_content = gpt_comments + '\n' + cleaned_observations

# Write to the output file
with open(output_path, 'w') as output_file:
    output_file.write(combined_content)

print(f'Combined content has been saved to {output_path}')

Combined content has been saved to data/prover-llm_v0/comments/comments.txt


Split observations into topic-wise observations

In [47]:
# Define file paths
observations_path = 'data/prover-llm_v0/comments/observations.txt'
algebra_path = 'data/prover-llm_v0/comments/algebra.txt'
combi_path = 'data/prover-llm_v0/comments/combi.txt'
nt_path = 'data/prover-llm_v0/comments/nt.txt'

# Function to remove numbering and clean content
def clean_section(content):
    # Remove numbering (like "1. ", "2. ", etc.)
    content = re.sub(r'^\d+\.\s+', '', content, flags=re.MULTILINE)
    # Remove extra blank lines but keep a single newline separation
    content = re.sub(r'\n\s*\n+', '\n', content).strip()
    return content

# Read the contents of the observations file
with open(observations_path, 'r') as file:
    observations = file.read()

# Split the text into sections based on the headers
# This assumes that sections are marked by headers like **Algebra**, **Combinatorics**, and **Number Theory**
algebra_content = re.search(r'\*\*Algebra\*\*(.*?)(?=\*\*|$)', observations, re.DOTALL)
combi_content = re.search(r'\*\*Combinatorics\*\*(.*?)(?=\*\*|$)', observations, re.DOTALL)
nt_content = re.search(r'\*\*Number Theory\*\*(.*?)(?=\*\*|$)', observations, re.DOTALL)

# Clean and write Algebra content to file
if algebra_content:
    cleaned_algebra = clean_section(algebra_content.group(1))
    with open(algebra_path, 'w') as algebra_file:
        algebra_file.write(cleaned_algebra)

# Clean and write Combinatorics content to file
if combi_content:
    cleaned_combi = clean_section(combi_content.group(1))
    with open(combi_path, 'w') as combi_file:
        combi_file.write(cleaned_combi)

# Clean and write Number Theory content to file
if nt_content:
    cleaned_nt = clean_section(nt_content.group(1))
    with open(nt_path, 'w') as nt_file:
        nt_file.write(cleaned_nt)

print(f'Files have been split and saved as {algebra_path}, {combi_path}, and {nt_path}')


Files have been split and saved as data/prover-llm_v0/comments/algebra.txt, data/prover-llm_v0/comments/combi.txt, and data/prover-llm_v0/comments/nt.txt


Pick the best comment from lean proof

In [33]:
# read comments as string from output_path
comments = open(output_path, 'r').read()
comments

"Carefully read and understand the problem statement.\nIdentify what is being asked and what is given.\nLook for patterns or regularities in the problem.\nConsider simplifying the problem or working with a special case.\nUse mathematical induction for proofs involving integers.\nApply the Pigeonhole Principle in counting problems.\nAssume the opposite to prove by contradiction.\nIdentify invariants that remain constant during transformations.\nBreak the problem into smaller, more manageable parts.\nConsider extreme values to test the boundaries of the problem.\nUse the contrapositive form of statements for proof.\nSimplify complex expressions through algebraic manipulation.\nApply well-known inequalities like AM ≥ GM or Cauchy-Schwarz.\nLook for opportunities to factor expressions.\nUse symmetry to reduce the complexity of the problem.\nCheck the parity (odd or even nature) of numbers involved.\nApply the Law of Sines or Cosines where appropriate.\nSet up equations based on the given c

In [48]:
alg_comments = open(algebra_path, 'r').read()

In [49]:
prompt = f"""This is the beginning of a proof of an algebra problem.
/-!
# International Mathematical Olympiad 1963, Problem 5

Prove that cos(π/7) - cos(2π/7) + cos(3π/7) = 1/2.
-/

namespace Imo1963P5

open scoped Real

problem imo1963_p5 :
    Real.cos (π/7) - Real.cos (2*π/7) + Real.cos (3*π/7) = 1/2 := by
  rw [show (2*π/7) = π - (5*π/7) by linarith]
  rw [Real.cos_pi_sub]
  simp only [sub_neg_eq_add]
  have h : 2 * Real.sin (π / 7) ≠ 0 := by
    simp only [ne_eq, mul_eq_zero, OfNat.ofNat_ne_zero, false_or]
    apply ne_of_gt
    apply Real.sin_pos_of_pos_of_lt_pi
    simp only [Nat.ofNat_pos, div_pos_iff_of_pos_right, Real.pi_pos]
    trans 1
    · rw [div_lt_one (by linarith only)]
      linarith only [Real.pi_le_four]
    · linarith only [Real.pi_gt_three]
  apply (mul_right_inj' h).mp
  rw [left_distrib, left_distrib]


Out of these comments, which one is most appropriate for guiding the remaining proof?
{alg_comments}
"""

In [50]:
len(prompt)

4393

In [51]:
len(tokenizer(prompt)['input_ids'])

1071

In [52]:
response, history = model.chat(tokenizer, prompt, history=[], meta_instruction="")
pprint(response)

('The most appropriate comment for guiding the remaining proof is:\n'
 '\n'
 'Recognize that substituting \\(y = kx\\) simplifies the equation.')


false but ok :P let's give it more info

In [53]:
prompt = """ Here's a proof
/-!
# International Mathematical Olympiad 1963, Problem 5

Prove that cos(π/7) - cos(2π/7) + cos(3π/7) = 1/2.
-/

namespace Imo1963P5

open scoped Real

problem imo1963_p5 :
    Real.cos (π/7) - Real.cos (2*π/7) + Real.cos (3*π/7) = 1/2 := by
  rw [show (2*π/7) = π - (5*π/7) by linarith]
  rw [Real.cos_pi_sub]
  simp only [sub_neg_eq_add]
  have h : 2 * Real.sin (π / 7) ≠ 0 := by
    simp only [ne_eq, mul_eq_zero, OfNat.ofNat_ne_zero, false_or]
    apply ne_of_gt
    apply Real.sin_pos_of_pos_of_lt_pi
    simp only [Nat.ofNat_pos, div_pos_iff_of_pos_right, Real.pi_pos]
    trans 1
    · rw [div_lt_one (by linarith only)]
      linarith only [Real.pi_le_four]
    · linarith only [Real.pi_gt_three]
  apply (mul_right_inj' h).mp
  rw [left_distrib, left_distrib]
  have prod_sum : ∀ (x y : ℝ),
      2 * Real.sin x * Real.cos y = Real.sin (x + y) - Real.sin (y - x) := by
    intro x y
    rw [Real.sin_add, Real.sin_sub]
    linarith only
  rw [prod_sum, prod_sum, prod_sum]
  rw [show (π / 7 + π / 7)     = 2 * π / 7 by linarith only]
  rw [show (π / 7 - π / 7)     = 0         by linarith only]
  rw [show (π / 7 + 5 * π / 7) = 6 * π / 7 by linarith only]
  rw [show (5 * π / 7 - π / 7) = 4 * π / 7 by linarith only]
  rw [show (π / 7 + 3 * π / 7) = 4 * π / 7 by linarith only]
  rw [show (3 * π / 7 - π / 7) = 2 * π / 7 by linarith only]
  rw [Real.sin_zero]
  ring_nf
  rw [← Real.sin_pi_sub]
  rw [show (π - π * (6 / 7)) = π / 7 by linarith]
  congr
  linarith
end Imo1963P5

After rw [prod_sum, prod_sum, prod_sum], what is the next thing the proof is doing? Give a hint
"""

In [54]:
response, history = model.chat(tokenizer, prompt, history=[], meta_instruction="")
pprint(response)

('The next thing the proof is doing is simplifying the expression using '
 'trigonometric identities and algebraic manipulation. The main identity used '
 'here is the double angle formula for sine, which states that sin(2x) = '
 '2sin(x)cos(x). By applying this formula to the terms of the expression, the '
 'proof is able to cancel out some terms and simplify the expression to 1/2.')


In [55]:
prompt = """ Here's a proof
/-!
# International Mathematical Olympiad 1963, Problem 5

Prove that cos(π/7) - cos(2π/7) + cos(3π/7) = 1/2.
-/

namespace Imo1963P5

open scoped Real

problem imo1963_p5 :
    Real.cos (π/7) - Real.cos (2*π/7) + Real.cos (3*π/7) = 1/2 := by
  rw [show (2*π/7) = π - (5*π/7) by linarith]
  rw [Real.cos_pi_sub]
  simp only [sub_neg_eq_add]
  have h : 2 * Real.sin (π / 7) ≠ 0 := by
    simp only [ne_eq, mul_eq_zero, OfNat.ofNat_ne_zero, false_or]
    apply ne_of_gt
    apply Real.sin_pos_of_pos_of_lt_pi
    simp only [Nat.ofNat_pos, div_pos_iff_of_pos_right, Real.pi_pos]
    trans 1
    · rw [div_lt_one (by linarith only)]
      linarith only [Real.pi_le_four]
    · linarith only [Real.pi_gt_three]
  apply (mul_right_inj' h).mp
  rw [left_distrib, left_distrib]
  have prod_sum : ∀ (x y : ℝ),
      2 * Real.sin x * Real.cos y = Real.sin (x + y) - Real.sin (y - x) := by
    intro x y
    rw [Real.sin_add, Real.sin_sub]
    linarith only
  rw [prod_sum, prod_sum, prod_sum]
  rw [show (π / 7 + π / 7)     = 2 * π / 7 by linarith only]
  rw [show (π / 7 - π / 7)     = 0         by linarith only]
  rw [show (π / 7 + 5 * π / 7) = 6 * π / 7 by linarith only]
  rw [show (5 * π / 7 - π / 7) = 4 * π / 7 by linarith only]
  rw [show (π / 7 + 3 * π / 7) = 4 * π / 7 by linarith only]
  rw [show (3 * π / 7 - π / 7) = 2 * π / 7 by linarith only]
  rw [Real.sin_zero]
  ring_nf
  rw [← Real.sin_pi_sub]
  rw [show (π - π * (6 / 7)) = π / 7 by linarith]
  congr
  linarith
end Imo1963P5

After rw [prod_sum, prod_sum, prod_sum], what is the next thing the proof is doing? Which of the following is most accurate:
{alg_comments}
"""
response, history = model.chat(tokenizer, prompt, history=[], meta_instruction="")
pprint(response)

('After rw [prod_sum, prod_sum, prod_sum], the next thing the proof is doing '
 'is to simplify the expression by adding and subtracting angles that are '
 'multiples of π/7, so that we can apply the sum and difference formulas for '
 'sine again. The proof is most accurately described by the following steps:\n'
 '\n'
 '1. Apply the sum and difference formulas for sine to the expression.\n'
 '2. Add and subtract multiples of π/7 to some of the terms, so that we can '
 'apply the sum and difference formulas for sine again.\n'
 '3. Simplify the expression using trigonometric identities and properties.\n'
 '4. Prove that the expression is equal to 1/2.\n'
 '\n'
 'So, the proof is most accurately described as "simplifying the expression by '
 'adding and subtracting angles that are multiples of π/7, so that we can '
 'apply the sum and difference formulas for sine again.".')


In [3]:
prompt = """ Here's a proof
/-!
# International Mathematical Olympiad 1963, Problem 5

Prove that cos(π/7) - cos(2π/7) + cos(3π/7) = 1/2.
-/

namespace Imo1963P5

open scoped Real

problem imo1963_p5 :
    Real.cos (π/7) - Real.cos (2*π/7) + Real.cos (3*π/7) = 1/2 := by
  rw [show (2*π/7) = π - (5*π/7) by linarith]
  rw [Real.cos_pi_sub]
  simp only [sub_neg_eq_add]
  have h : 2 * Real.sin (π / 7) ≠ 0 := by
    simp only [ne_eq, mul_eq_zero, OfNat.ofNat_ne_zero, false_or]
    apply ne_of_gt
    apply Real.sin_pos_of_pos_of_lt_pi
    simp only [Nat.ofNat_pos, div_pos_iff_of_pos_right, Real.pi_pos]
    trans 1
    · rw [div_lt_one (by linarith only)]
      linarith only [Real.pi_le_four]
    · linarith only [Real.pi_gt_three]
  apply (mul_right_inj' h).mp
  rw [left_distrib, left_distrib]
  have prod_sum : ∀ (x y : ℝ),
      2 * Real.sin x * Real.cos y = Real.sin (x + y) - Real.sin (y - x) := by
    intro x y
    rw [Real.sin_add, Real.sin_sub]
    linarith only
  rw [prod_sum, prod_sum, prod_sum]
  rw [show (π / 7 + π / 7)     = 2 * π / 7 by linarith only]
  rw [show (π / 7 - π / 7)     = 0         by linarith only]
  rw [show (π / 7 + 5 * π / 7) = 6 * π / 7 by linarith only]
  rw [show (5 * π / 7 - π / 7) = 4 * π / 7 by linarith only]
  rw [show (π / 7 + 3 * π / 7) = 4 * π / 7 by linarith only]
  rw [show (3 * π / 7 - π / 7) = 2 * π / 7 by linarith only]
  rw [Real.sin_zero]
  ring_nf
  rw [← Real.sin_pi_sub]
  rw [show (π - π * (6 / 7)) = π / 7 by linarith]
  congr
  linarith
end Imo1963P5

After rw [prod_sum, prod_sum, prod_sum], what is the next thing the proof is doing? Which of the following is most accurate. Choose exactly one from this list, verbatim:
{alg_comments[:len(alg_comments) // 5]}
"""
response, history = model.chat(tokenizer, prompt, history=[], meta_instruction="")
pprint(response)

KeyboardInterrupt: 

It doesn't follow directions. Try GPT 4

In [6]:
prompt = """ Here's a proof
/-!
# International Mathematical Olympiad 1963, Problem 5

Prove that cos(π/7) - cos(2π/7) + cos(3π/7) = 1/2.
-/

namespace Imo1963P5

open scoped Real

problem imo1963_p5 :
    Real.cos (π/7) - Real.cos (2*π/7) + Real.cos (3*π/7) = 1/2 := by
  rw [show (2*π/7) = π - (5*π/7) by linarith]
  rw [Real.cos_pi_sub]
  simp only [sub_neg_eq_add]
  have h : 2 * Real.sin (π / 7) ≠ 0 := by
    simp only [ne_eq, mul_eq_zero, OfNat.ofNat_ne_zero, false_or]
    apply ne_of_gt
    apply Real.sin_pos_of_pos_of_lt_pi
    simp only [Nat.ofNat_pos, div_pos_iff_of_pos_right, Real.pi_pos]
    trans 1
    · rw [div_lt_one (by linarith only)]
      linarith only [Real.pi_le_four]
    · linarith only [Real.pi_gt_three]
  apply (mul_right_inj' h).mp
  rw [left_distrib, left_distrib]
  have prod_sum : ∀ (x y : ℝ),
      2 * Real.sin x * Real.cos y = Real.sin (x + y) - Real.sin (y - x) := by
    intro x y
    rw [Real.sin_add, Real.sin_sub]
    linarith only

This is the remainder of the proof. What is the next step? 

  rw [prod_sum, prod_sum, prod_sum]
  rw [show (π / 7 + π / 7)     = 2 * π / 7 by linarith only]
  rw [show (π / 7 - π / 7)     = 0         by linarith only]
  rw [show (π / 7 + 5 * π / 7) = 6 * π / 7 by linarith only]
  rw [show (5 * π / 7 - π / 7) = 4 * π / 7 by linarith only]
  rw [show (π / 7 + 3 * π / 7) = 4 * π / 7 by linarith only]
  rw [show (3 * π / 7 - π / 7) = 2 * π / 7 by linarith only]
  rw [Real.sin_zero]
  ring_nf
  rw [← Real.sin_pi_sub]
  rw [show (π - π * (6 / 7)) = π / 7 by linarith]
  congr
  linarith
"""
response, history = model.chat(tokenizer, prompt, history=[], meta_instruction="")
pprint(response)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


('The next step is to simplify the expression using trigonometric identities '
 'and algebraic manipulation. We have already established the identity $2\\sin '
 'x\\cos y = \\sin(x+y) - \\sin(y-x)$, which will be used to rewrite the '
 'expression involving the cosine of angles. Then, we will use the fact that '
 '$\\sin(\\pi - x) = \\sin x$ to further simplify the expression. Finally, we '
 'will use the fact that $\\sin(\\pi - x) = \\sin x$ again to rewrite the '
 'expression in terms of sines of angles that are multiples of $\\pi/7$. The '
 'final step will be to simplify the expression using algebraic manipulation '
 'and the fact that $\\sin(\\pi - x) = \\sin x$.')


Get logprobs of the comments

In [5]:
comments = open("/home/ubuntu/ohm-tree-filesys/plasma-converter/data/prover-llm_v0/comments/comments_v2.txt", 'r').read().split("\n")

In [7]:
device = "cuda"

p = []

for i in range(len(comments)):
    target = comments[i]

    # Tokenize the prompt and target separately
    prompt_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    target_ids = tokenizer.encode(target, return_tensors='pt').to(device)

    # Concatenate the prompt and target ids
    input_ids = torch.cat([prompt_ids, target_ids[:, 1:]], dim=-1)  # Exclude the first token of target_ids if it's a special token like BOS

    # Get the length of the prompt to know where the target starts
    prompt_length = prompt_ids.size(1)

    # Run the model to get logits
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits

    # Compute log probabilities
    log_probs = torch.log_softmax(logits, dim=-1)

    # Extract log probabilities for the target tokens
    target_log_probs = []
    for i in range(prompt_length, input_ids.size(1)):
        # The token ID at the current position
        token_id = input_ids[0, i]
        # Log probability of the token given the previous tokens
        token_log_prob = log_probs[0, i - 1, token_id]
        target_log_probs.append(token_log_prob.item())

    # Sum the log probabilities to get the total log probability of the target
    total_log_prob = sum(target_log_probs)

    # print("Log probabilities of the target tokens:", target_log_probs)
    # print(f"Cum. log probability of {target}:", total_log_prob)
    p.append((target, total_log_prob))

In [8]:
# sort list of tuples by the second item
p.sort(key=lambda x: x[1])

[('Notice that substituting \\(x = y + k\\) aligns variables for easier manipulation.',
  -138.91053668706445),
 ('Employ combinatorial arguments in counting scenarios.',
  -106.41187334060646),
 ('Apply the concept that the sum of two primes is even, except for the case involving 2.',
  -106.37579877430107),
 ('Use Euler’s Totient Function to find numbers coprime to a given integer.',
  -102.45052352607667),
 ('Check the parity (odd or even nature) of numbers involved.',
  -94.54273155960004),
 ('Observe that adding a specific term completes a known identity.',
  -93.939628752938),
 ('Use properties of quadratic residues to solve the equation modulo a prime.',
  -91.52527168906272),
 ('Apply Legendre’s Symbol to determine quadratic residues.',
  -89.92388277923851),
 ("Observe that substituting \\(t = x^2\\) reduces the equation's degree.",
  -89.22910634544678),
 ('Identify that applying the Euclidean algorithm finds the greatest common divisor (GCD).',
  -87.57252411528862),
 ('Iden

In [19]:
print("\n".join(comments))

Look for patterns or regularities in the problem.
Consider simplifying the problem or working with a special case.
Use mathematical induction for proofs involving integers.
Assume the opposite to prove by contradiction.
Identify invariants that remain constant during transformations.
Break the problem into smaller, more manageable parts.
Use the contrapositive form of statements for proof.
Simplify complex expressions through algebraic manipulation.
Apply Cauchy-Schwarz.
Look for opportunities to factor expressions.
Use symmetry to reduce the complexity of the problem.
Check the parity (odd or even nature) of numbers involved.
Apply the Law of Sines or Cosines where appropriate.
Set up equations based on the given conditions.
Explore recursive patterns in sequences.
Use Vieta's formulas relating roots and coefficients of polynomials.
Apply modular arithmetic in number theory problems.
Employ combinatorial arguments in counting scenarios.
Use the Principle of Inclusion-Exclusion in over

In [20]:
combined_comments = '\n'.join(comments)

In [23]:
prompt = f"""
Which of the following sentences is most similar to 

'The next step is to simplify the expression using trigonometric identities '
 'and algebraic manipulation. We have already established the identity $2\\sin '
 'x\\cos y = \\sin(x+y) - \\sin(y-x)$, which will be used to rewrite the '
 'expression involving the cosine of angles. Then, we will use the fact that '
 '$\\sin(\\pi - x) = \\sin x$ to further simplify the expression. Finally, we '
 'will use the fact that $\\sin(\\pi - x) = \\sin x$ again to rewrite the '
 'expression in terms of sines of angles that are multiples of $\\pi/7$. The '
 'final step will be to simplify the expression using algebraic manipulation '
 'and the fact that $\\sin(\\pi - x) = \\sin x$.'?

 {combined_comments}

# Output the comment verbatim from the list:
"""
response, history = model.chat(tokenizer, prompt, history=[], meta_instruction="")
pprint(response)

("'The next step is to simplify the expression using trigonometric identities "
 'and algebraic manipulation. We have already established the identity $2\\sin '
 'x\\cos y = \\sin(x+y) - \\sin(y-x)$, which will be used to rewrite the '
 'expression involving the cosine of angles. Then, we will use the fact that '
 '$\\sin(\\pi - x) = \\sin x$ to further simplify the expression. Finally, we '
 'will use the fact that $\\sin(\\pi - x) = \\sin x$ again to rewrite the '
 'expression in terms of sines of angles that are multiples of $\\pi/7$. The '
 'final step will be to simplify the expression using algebraic manipulation '
 'and the fact that $\\sin(\\pi - x) = \\sin x$.')


In [15]:
prompt

'\nWhich of the following sentences is most similar to \n\n\'The next step is to simplify the expression using trigonometric identities \'\n \'and algebraic manipulation. We have already established the identity $2\\sin \'\n \'x\\cos y = \\sin(x+y) - \\sin(y-x)$, which will be used to rewrite the \'\n \'expression involving the cosine of angles. Then, we will use the fact that \'\n \'$\\sin(\\pi - x) = \\sin x$ to further simplify the expression. Finally, we \'\n \'will use the fact that $\\sin(\\pi - x) = \\sin x$ again to rewrite the \'\n \'expression in terms of sines of angles that are multiples of $\\pi/7$. The \'\n \'final step will be to simplify the expression using algebraic manipulation \'\n \'and the fact that $\\sin(\\pi - x) = \\sin x$.\'?\n\n [\'Look for patterns or regularities in the problem.\', \'Consider simplifying the problem or working with a special case.\', \'Use mathematical induction for proofs involving integers.\', \'Assume the opposite to prove by contradict

In [25]:
prompt = """ Here is a question, presented with informal langage and formal Lean 4 code.

/-!
# International Mathematical Olympiad 1963, Problem 5

Prove that cos(π/7) - cos(2π/7) + cos(3π/7) = 1/2.
-/

```lean4
namespace Imo1963P5

open scoped Real

problem imo1963_p5 :
    Real.cos (π/7) - Real.cos (2*π/7) + Real.cos (3*π/7) = 1/2 := by
```

A student is attempting to solve this problem, and has gotten this far:

```lean4
  rw [show (2*π/7) = π - (5*π/7) by linarith]
  rw [Real.cos_pi_sub]
  simp only [sub_neg_eq_add]
  have h : 2 * Real.sin (π / 7) ≠ 0 := by
    simp only [ne_eq, mul_eq_zero, OfNat.ofNat_ne_zero, false_or]
    apply ne_of_gt
    apply Real.sin_pos_of_pos_of_lt_pi
    simp only [Nat.ofNat_pos, div_pos_iff_of_pos_right, Real.pi_pos]
    trans 1
    · rw [div_lt_one (by linarith only)]
      linarith only [Real.pi_le_four]
    · linarith only [Real.pi_gt_three]
  apply (mul_right_inj' h).mp
  rw [left_distrib, left_distrib]
  have prod_sum : ∀ (x y : ℝ),
      2 * Real.sin x * Real.cos y = Real.sin (x + y) - Real.sin (y - x) := by
    intro x y
    rw [Real.sin_add, Real.sin_sub]
    linarith only
  rw [prod_sum, prod_sum, prod_sum]
```

Summarize what has been proven. Please suggest 5 distinct proof ideas which can finish from this point. Be sure to delimit your ideas with <IDEA> </IDEA>.
"""
response, history = model.chat(tokenizer, prompt, history=[], meta_instruction="")
pprint(response)

('1. <IDEA> Use the product-to-sum identities to simplify the expression and '
 'then apply the sum-to-product identity to find the value of the expression. '
 '</IDEA>\n'
 '2. <IDEA> Use the angle addition formula for sine to rewrite the expression '
 'and then use the fact that $\\sin(4\\pi/7) = -\\sin(3\\pi/7)$ and '
 '$\\sin(5\\pi/7) = -\\sin(\\pi/7)$ to simplify the expression. </IDEA>\n'
 '3. <IDEA> Use the angle subtraction formula for sine to rewrite the '
 'expression and then use the fact that $\\sin(4\\pi/7) = -\\sin(3\\pi/7)$ and '
 '$\\sin(5\\pi/7) = -\\sin(\\pi/7)$ to simplify the expression. </IDEA>\n'
 '4. <IDEA> Use the angle addition formula for sine to rewrite the expression '
 'and then use the fact that $\\sin(4\\pi/7) = -\\sin(3\\pi/7)$ and '
 '$\\sin(5\\pi/7) = -\\sin(\\pi/7)$ to simplify the expression. </IDEA>\n'
 '5. <IDEA> Use the angle subtraction formula for sine to rewrite the '
 'expression and then use the fact that $\\sin(4\\pi/7) = -\\sin(3\\pi/7)$ and

In [ ]:
prompt = """ Here is a question, presented with informal langage and formal Lean 4 code.

/-!
# International Mathematical Olympiad 1963, Problem 5

Prove that cos(π/7) - cos(2π/7) + cos(3π/7) = 1/2.
-/

```lean4
namespace Imo1963P5

open scoped Real

problem imo1963_p5 :
    Real.cos (π/7) - Real.cos (2*π/7) + Real.cos (3*π/7) = 1/2 := by
```

A student is attempting to solve this problem, and has gotten this far:

```lean4
  rw [show (2*π/7) = π - (5*π/7) by linarith]
  rw [Real.cos_pi_sub]
  simp only [sub_neg_eq_add]
  have h : 2 * Real.sin (π / 7) ≠ 0 := by
    simp only [ne_eq, mul_eq_zero, OfNat.ofNat_ne_zero, false_or]
    apply ne_of_gt
    apply Real.sin_pos_of_pos_of_lt_pi
    simp only [Nat.ofNat_pos, div_pos_iff_of_pos_right, Real.pi_pos]
    trans 1
    · rw [div_lt_one (by linarith only)]
      linarith only [Real.pi_le_four]
    · linarith only [Real.pi_gt_three]
  apply (mul_right_inj' h).mp
  rw [left_distrib, left_distrib]
  have prod_sum : ∀ (x y : ℝ),
      2 * Real.sin x * Real.cos y = Real.sin (x + y) - Real.sin (y - x) := by
    intro x y
    rw [Real.sin_add, Real.sin_sub]
    linarith only
  rw [prod_sum, prod_sum, prod_sum]
```

At this point, the student is considering the following possible ways to finish the proof:

1. Use the product-to-sum identities to simplify the expression and '
 'then apply the sum-to-product identity to find the value of the expression.
2. Use the angle addition formula for sine to rewrite the expression and then use the fact that $\\sin(4\\pi/7) = -\\sin(3\\pi/7)$ and $\\sin(5\\pi/7) = -\\sin(\\pi/7)$ to simplify the expression. </IDEA>
3. <IDEA> Use the angle subtraction formula for sine to rewrite the expression and then use the fact that $\\sin(4\\pi/7) = -\\sin(3\\pi/7)$ and '
 '$\\sin(5\\pi/7) = -\\sin(\\pi/7)$ to simplify the expression. </IDEA>\n'
 '4. <IDEA> Use the angle addition formula for sine to rewrite the expression '
 'and then use the fact that $\\sin(4\\pi/7) = -\\sin(3\\pi/7)$ and '
 '$\\sin(5\\pi/7) = -\\sin(\\pi/7)$ to simplify the expression. </IDEA>\n'
 '5. <IDEA> Use the angle subtraction formula for sine to rewrite the '
 'expression and then use the fact that $\\sin(4\\pi/7) = -\\sin(3\\pi/7)$ and '
 '$\\sin(5\\pi/7) = -\\sin(\\pi/7)$ to simplify the expression. </IDEA>\n

Here is the correct completion to this proof:
```lean4
  rw [show (π / 7 + π / 7)     = 2 * π / 7 by linarith only]
  rw [show (π / 7 - π / 7)     = 0         by linarith only]
  rw [show (π / 7 + 5 * π / 7) = 6 * π / 7 by linarith only]
  rw [show (5 * π / 7 - π / 7) = 4 * π / 7 by linarith only]
  rw [show (π / 7 + 3 * π / 7) = 4 * π / 7 by linarith only]
  rw [show (3 * π / 7 - π / 7) = 2 * π / 7 by linarith only]
  rw [Real.sin_zero]
  ring_nf
  rw [← Real.sin_pi_sub]
  rw [show (π - π * (6 / 7)) = π / 7 by linarith]
  congr
  linarith
end Imo1963P5
```

Please first give feedback on each idea.

Then, after you are finished giving feedback, give a confidence score for each idea, which should be a number from 0 to 100. For example,
<CONFIDENCE>
1. 11
2. 32
3. 61
4. 05
5. 99
</CONFIDENCE>
"""
response, history = model.chat(tokenizer, prompt, history=[], meta_instruction="")
pprint(response)



In [26]:
# print number of tokens in prompt
len(tokenizer(prompt)['input_ids'])

460

In [29]:
#print the tokens mapped to the input_ids
tokenizer.convert_ids_to_tokens(tokenizer(prompt)['input_ids'])

['<s>',
 '▁Here',
 '▁is',
 '▁a',
 '▁question',
 ',',
 '▁presented',
 '▁with',
 '▁informal',
 '▁lang',
 'age',
 '▁and',
 '▁formal',
 '▁Le',
 'an',
 '▁',
 '4',
 '▁code',
 '.\n\n',
 '/-',
 '!\n',
 '#',
 '▁International',
 '▁Math',
 'ematic',
 'al',
 '▁Olymp',
 'i',
 'ad',
 '▁',
 '196',
 '3',
 ',',
 '▁Problem',
 '▁',
 '5',
 '\n\n',
 'Pro',
 've',
 '▁that',
 '▁cos',
 '(',
 'π',
 '/',
 '7',
 ')',
 '▁-',
 '▁cos',
 '(',
 '2',
 'π',
 '/',
 '7',
 ')',
 '▁+',
 '▁cos',
 '(',
 '3',
 'π',
 '/',
 '7',
 ')',
 '▁=',
 '▁',
 '1',
 '/',
 '2',
 '.\n',
 '-',
 '/\n\n',
 '``',
 '`',
 'lean',
 '4',
 '\n',
 'namespace',
 '▁Im',
 'o',
 '196',
 '3',
 'P',
 '5',
 '\n\n',
 'open',
 '▁scoped',
 '▁Real',
 '\n\n',
 'problem',
 '▁im',
 'o',
 '196',
 '3',
 '_p',
 '5',
 '▁:\n',
 '▁▁▁▁',
 'Real',
 '.cos',
 '▁(',
 'π',
 '/',
 '7',
 ')',
 '▁-',
 '▁Real',
 '.cos',
 '▁(',
 '2',
 '*',
 'π',
 '/',
 '7',
 ')',
 '▁+',
 '▁Real',
 '.cos',
 '▁(',
 '3',
 '*',
 'π',
 '/',
 '7',
 ')',
 '▁=',
 '▁',
 '1',
 '/',
 '2',
 '▁:=',
 '▁by',
 '\n